In [14]:
# My own try with Backward Euler method.
# Model predator-prey population dynamics
#
#    dr/dt =  a r - b r f
#    df/dt = -m f + c b r f 
#
# where: r  = population of prey (e.g. number of rabbits)
#        f  = population of predator (e.g. number of foxes)
#        a  = birth rate of prey per capita
#        m  = mortality rate of predator per capita
#        b  = predation rate per prey x predator
#        c  = frequency at which predation results prey

import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
import time
# Main for solving predator-prey model using Backward Euler
p = {}
p['a'] = 2.0
p['b'] = 0.01
p['c'] = 0.1
p['m'] = 1.0

rI = 500
fI = 100     
tF = 10.0 # final time to simulate to (month)
dt = 0.1 # time increment to give solutions at (s)

# Calculate residual function to be equal to 0.
un = np.array([0, 0])
tn = 0
def evalr(v):
    """Calculates the residual function to find the value of u1.

    Args:
        u (np.array): current state vector
        t (float): current time
        dt (timestep): timestep value

    Returns:
        r (function): residual function whose parameters
                    depend on u, t, dt.
    """
    return (v - un)/dt - evalf(v, tn + dt)


# Calculate forcing function (du_/dt) = f_(u_, t)
def evalf(u, t):
    """Calculates current value of f_(u_, t)

    Args:
        u (np.array): current state vector with 2 elements. u_[0] = r, u_[1] = f
        t (float): current time
        
    Returns:
        f (np.array): current value of du_/dt vector. 
                      du[0]/dt = f[0], du[1]/dt = f[1]
    """
    a = p['a']
    b = p['b']
    c = p['c']
    m = p['m']
    
    f = np.zeros(2)
    f[0] = a*u[0] - b*u[0]*u[1]
    f[1] = -m*u[1] + c*b*u[0]*u[1]
    
    return f

# Calculate Jacobian.
def evalJ(u):
    """ Calculates the Jacobian matrix of the function f_(u_, t)
        for the current state.

    Args:
        u (np.array): current state vector.
        f (np.array): current value of the forcing function.
    
    Returns:
        J (np.array): the Jacobian matrix.
    """
    a = p['a']
    b = p['b']
    c = p['c']
    m = p['m']
    
    J = np.eye(len(u))
    J[0][0] = a - b*u[1]
    J[1][0] = c*b*u[1]
    J[0][1] = -b*u[0]
    J[1][1] = -m + c*b*u[0]
    
    return J




dt = 0.01
tI = 0  # Initial time
tF = 10  # Final time
uI = np.array([500, 50])  # [# rabbits, # foxes]
t = np.arange(tI, tF, dt)
u = np.empty([len(t), len(uI)])
def BackwardEuler_solve(tI, tF, evalr, evalf, evalJ, uI, dt):
    for i in range(len(t)):
        tn = t[i]
        un = u[i]
        
        un1 = scipy.optimize.root(evalr, un, jac = evalJ)
        u[i+1] = un1
        
        return u

u_ = BackwardEuler_solve(tI, tF, evalr, evalf, evalJ, uI, dt)

fig, ax = plt.subplots()
ax.plot(t, u_[:, 0], label = "rabbits", color = "blue")
ax.plot(t, u_[:, 1], label = "foxes", color = "red")

TypeError: evalJ() missing 1 required positional argument: 'f'